# Importing packages. 

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from datetime import timedelta

# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Data display coustomization
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 120)

# module for performing test train split
from sklearn.model_selection import train_test_split

# Module for feture scaling
from sklearn.preprocessing import MinMaxScaler

# statmodel linear regression
import statsmodels.api as sm

# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Reading the datasets. 

In [2]:
# reading loan level data.
loan_df = pd.read_excel('loan_level.xlsx')
repay_df = pd.read_excel('repayment_data.xlsx')
cust_df = pd.read_excel('customer_level.xlsx')

In [3]:
# Merge loan_data and repayment_data with the common columns 'partner_loan_id_comb'.
df_final = pd.merge(loan_df, repay_df, on='partner_loan_id_comb', how='left')
df_final.head()

,partner_loan_id_comb,borrower_pan,loan_prin_amt,loan_proc_fee,loan_amt_disbursed,loan_int_rt,loan_int_amt,loan_emi_dte_1,loan_end_dte,loan_disbursement_upd_dte,applied_amount,emi_amt,net_disbur_amt,old_partnerid,repayment_date,repayment_amt
0,KUD-BTH-781116899997182949199636,BAMPC1949R,2100,545.16,1554.84,0.0063,13.23,2020-11-29,2020-12-05,2020-11-29,2100,2113.23,1554.84,KUD-BTH-78111,NaN,NaN
1,KUD-BTH-781116900112694658520768,BPJPM5784E,1800,467.28,1332.72,0.0063,11.34,2020-11-29,2020-12-05,2020-11-29,1800,1811.34,1332.72,KUD-BTH-78111,NaN,NaN
2,KUD-BTH-781116900154773213910750,BCYPK0940G,1800,467.28,1332.72,0.0063,11.34,2020-11-29,2020-12-05,2020-11-29,1800,1811.34,1332.72,KUD-BTH-78111,NaN,NaN
3,KUD-BTH-781116900155739556417298,FWIPS4293A,4300,1116.28,3183.72,0.0063,27.09,2020-11-29,2020-12-05,2020-11-29,4300,4327.09,3183.72,KUD-BTH-78111,NaN,NaN
4,KUD-BTH-781116900171038510913233,BRTPD0471R,1800,467.28,1332.72,0.0063,11.34,2020-11-29,2020-12-05,2020-11-29,1800,1811.34,1332.72,KUD-BTH-78111,NaN,NaN


In [4]:
df = df_final[:]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 196755 entries, 0 to 196754
Data columns (total 16 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   partner_loan_id_comb       196755 non-null  object        
 1   borrower_pan               196755 non-null  object        
 2   loan_prin_amt              196755 non-null  int64         
 3   loan_proc_fee              196755 non-null  float64       
 4   loan_amt_disbursed         196755 non-null  float64       
 5   loan_int_rt                196755 non-null  float64       
 6   loan_int_amt               196755 non-null  float64       
 7   loan_emi_dte_1             196755 non-null  datetime64[ns]
 8   loan_end_dte               196755 non-null  datetime64[ns]
 9   loan_disbursement_upd_dte  196755 non-null  datetime64[ns]
 10  applied_amount             196755 non-null  int64         
 11  emi_amt                    196755 non-null  float64 

In [6]:
# Checking missing value percentage of each field before handling it . 
round(100 * (df.isnull().sum()/len(df.index)),2)

partner_loan_id_comb          0.00
borrower_pan                  0.00
loan_prin_amt                 0.00
loan_proc_fee                 0.00
loan_amt_disbursed            0.00
loan_int_rt                   0.00
loan_int_amt                  0.00
loan_emi_dte_1                0.00
loan_end_dte                  0.00
loan_disbursement_upd_dte     0.00
applied_amount                0.00
emi_amt                       0.00
net_disbur_amt                0.00
old_partnerid                 0.00
repayment_date               30.23
repayment_amt                30.23
dtype: float64

In [7]:
# Correcting data types of few fields.            
df['loan_prin_amt'] = df['loan_prin_amt'].astype(float)
df['repayment_date']= pd.to_datetime(df['repayment_date'])

# dropping few repeated columns. 
df = df.drop(['loan_int_rt','applied_amount','net_disbur_amt'],1)

In [8]:
df.describe()

,loan_prin_amt,loan_proc_fee,loan_amt_disbursed,loan_int_amt,emi_amt,repayment_amt
count,196755.000000,196755.000000,196755.000000,196755.000000,196755.000000,137281.000000
mean,2826.981271,721.463587,2105.517684,19.191640,2846.172911,2758.099388
std,1086.951109,280.898570,807.880439,7.169837,1093.989522,1002.022664
min,1000.000000,236.000000,740.400000,6.300000,1006.300000,800.000000
25%,1800.000000,467.280000,1332.720000,11.340000,1811.340000,2012.600000
50%,2600.000000,674.960000,1925.040000,17.500000,2616.380000,2616.380000
75%,3500.000000,908.600000,2665.440000,24.500000,3524.500000,3524.500000
max,7800.000000,2024.880000,5775.120000,54.600000,7854.600000,15358.000000


# Calculation of DPD variables and creating all the loan vaiables. 

In [9]:
# creating a column that holds current date in real time. 
df['current_date'] = pd.to_datetime(pd.datetime.now().date())


##  INITIAL DPD. 

# creation of initial_DPD metric. 
df['initial_DPD'] = df['repayment_date'] - df['loan_end_dte']

# computing null vlaues for initial_DPD. 
df['initial_DPD'] = df['initial_DPD'].fillna(df['current_date'] - df['loan_end_dte'])

# converting the initial_dpd column to numeric type for easier operations. 
df['initial_DPD'] = pd.to_numeric(df['initial_DPD'].dt.days, downcast='integer')


# creating a new column initial_DPD_new according 

df['initial_DPD_new'] = 0
for i in range(len(df['initial_DPD'].index)):
    if df['initial_DPD'].loc[i] >= 0:
        df['initial_DPD_new'].loc[i] = df['initial_DPD'].loc[i]
    else: 
        df['initial_DPD_new'].loc[i] = 0


In [10]:
#### DPD_Amount

# Initializing the new DPD_Amount column. 
df['DPD_Amount'] = 0

# specifying the date format type.
date_format_str = '%Y-%m-%d'

for i in range(len(df['initial_DPD'].index)):
    if df['initial_DPD_new'].loc[i] > 0 and df['loan_disbursement_upd_dte'].loc[i] <= datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['DPD_Amount'].loc[i] = (df['loan_prin_amt'].loc[i] * 0.03) + (df['loan_prin_amt'].loc[i] * 0.002 *(df['initial_DPD_new'].loc[i]-1))
    if df['initial_DPD_new'].loc[i] > 0 and df['initial_DPD_new'].loc[i] <= 30 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['DPD_Amount'].loc[i] = df['loan_prin_amt'].loc[i] * 0.01 * df['initial_DPD_new'].loc[i]
    if df['initial_DPD_new'].loc[i] > 30 and df['initial_DPD_new'].loc[i] <= 60 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['DPD_Amount'].loc[i] = ((df['loan_prin_amt'].loc[i] * 0.01 * 30) + df['loan_prin_amt'].loc[i] * 0.005 * (df['initial_DPD_new'].loc[i]-30))
    if df['initial_DPD_new'].loc[i] > 60 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():  
        df['DPD_Amount'].loc[i] = ((df['loan_prin_amt'].loc[i] * 0.01 * 30)+(df['loan_prin_amt'].loc[i] * 0.005 * 30)+(df['loan_prin_amt'].loc[i] * 0.002 * (df['initial_DPD_new'].loc[i] - 60)))

In [11]:
#### capped_dpd_amt

df['capped_dpd_amt'] = 0

for i in range(len(df['capped_dpd_amt'].index)):
    if df['DPD_Amount'].loc[i] > df['loan_prin_amt'].loc[i]:
        df['capped_dpd_amt'].loc[i] = df['loan_prin_amt'].loc[i]
    else:
        df['capped_dpd_amt'].loc[i] = df['DPD_Amount'].loc[i]
    
## expected_amt.. 

df['expected_amt'] = df['loan_prin_amt'] + df['loan_int_amt'] + df['capped_dpd_amt']


# calculating total repaid amount. 
# if repayment amount is null then we will impute the total repaid amount to 0 as no loan was repaid. 

df['total_repaid_amt'] = df['repayment_amt']
df['total_repaid_amt'] = df['total_repaid_amt'].fillna(0)

# calculating due amount.. 
df['due_amt'] = df['expected_amt'] - df['total_repaid_amt']

In [12]:
# Repayment Flag imputation. 

df['repayment_flag'] = ''

for i in range(len(df['repayment_flag'])):
    if df['due_amt'].loc[i] <= 10.00:
        df['repayment_flag'].loc[i] = 'Full'
    elif df['due_amt'].loc[i] > 10.00 and df['total_repaid_amt'].loc[i] != 0: 
        df['repayment_flag'].loc[i]= 'Partial'
    elif df['due_amt'].loc[i] > 10.00 and df['total_repaid_amt'].loc[i] == 0: 
        df['repayment_flag'].loc[i] = 'Unpaid'
    else: 
        continue

In [13]:
# Filling Repayment amount to 0 if there is no repayment done. 
df['repayment_amt'] = df['repayment_amt'].fillna(0)

In [14]:
# Calculating Final_DPD columns. 

df['final_dpd'] = 0
df['final_dpd_new'] = 0
df['final_dpd_amt'] = 0.0
df['final_capped_dpd_amt'] = 0.0
df['final_expected_amt'] = 0.0
df['final_due_amt'] = 0.0


for i in range(len(df['final_dpd'].index)):
    if df['repayment_flag'].loc[i] == 'Unpaid' or df['repayment_flag'].loc[i] == 'Full':
        df['final_dpd'].loc[i] = df['initial_DPD'].loc[i]
        df['final_dpd_new'].loc[i] = df['initial_DPD_new'].loc[i]
        df['final_dpd_amt'].loc[i] = df['DPD_Amount'].loc[i]
        df['final_capped_dpd_amt'].loc[i] = df['capped_dpd_amt'].loc[i]
        df['final_expected_amt'].loc[i] = df['expected_amt'].loc[i]
        df['final_due_amt'].loc[i] = df['due_amt'].loc[i]
    if df['repayment_flag'].loc[i] == 'Partial':
        df['final_dpd'].loc[i] = (df['current_date'].loc[i] - df['loan_end_dte'].loc[i]).days

# calculation of final_dpd_new according to repayment_flag and final_dpd.
for i in range(len(df['final_dpd'].index)):
    if df['repayment_flag'].loc[i] == 'Partial':
        if df['final_dpd'].loc[i] >= 0:
            df['final_dpd_new'].loc[i] = df['final_dpd'].loc[i]
        else:
            df['final_dpd_new'].loc[i] = 0

In [15]:
for i in range(len(df['final_dpd'].index)): 
    if df['final_dpd_new'].loc[i] > 0 and df['loan_disbursement_upd_dte'].loc[i] <= datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['final_dpd_amt'].loc[i] = (df['loan_prin_amt'].loc[i] * 0.03) + (df['loan_prin_amt'].loc[i] * 0.002 *(df['final_dpd_new'].loc[i]-1))
    if df['final_dpd_new'].loc[i] > 0 and df['final_dpd_new'].loc[i] <= 30 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['final_dpd_amt'].loc[i] = df['loan_prin_amt'].loc[i] * 0.01 * df['final_dpd_new'].loc[i]
    if df['final_dpd_new'].loc[i] > 30 and df['final_dpd_new'].loc[i] <= 60 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():
        df['final_dpd_amt'].loc[i] = ((df['loan_prin_amt'].loc[i] * 0.01 * 30) + df['loan_prin_amt'].loc[i] * 0.005 * (df['final_dpd_new'].loc[i]-30))
    if df['final_dpd_new'].loc[i] > 60 and df['loan_disbursement_upd_dte'].loc[i] > datetime.datetime.strptime('2020-08-18',date_format_str).date():  
        df['final_dpd_amt'].loc[i] = ((df['loan_prin_amt'].loc[i] * 0.01 * 30)+(df['loan_prin_amt'].loc[i] * 0.005 * 30)+(df['loan_prin_amt'].loc[i] * 0.002 * (df['final_dpd_new'].loc[i] - 60)))

In [16]:
# Calculation of final_capped_dpd_amt for partially paid loans.

for i in range(len(df['final_capped_dpd_amt'].index)):
    if df['repayment_flag'].loc[i] == 'Partial':
        if df['final_dpd_amt'].loc[i] > df['loan_prin_amt'].loc[i]: 
            df['final_capped_dpd_amt'].loc[i] = df['loan_prin_amt'].loc[i]
        else:
            df['final_capped_dpd_amt'].loc[i] = df['final_dpd_amt'].loc[i]

# Calculation of final_expected_amt for partially paid loans.  

for i in range(len(df['final_expected_amt'].index)):
    if df['repayment_flag'].loc[i] == 'Partial':
        df['final_expected_amt'].loc[i] = df['loan_prin_amt'].loc[i] + df['final_capped_dpd_amt'].loc[i] + df['loan_int_amt'].loc[i]

# Calculation of final_due_amt for partially paid loans. 

for i in range(len(df['final_due_amt'].index)):
    if df['repayment_flag'].loc[i] == 'Partial':
        df['final_due_amt'].loc[i] = df['final_expected_amt'].loc[i] - df['total_repaid_amt'].loc[i]

### Calculation of metrics related to risk control API.
    
    These variable are customer level or PAN level variables. 
1. total_loan_count
2. sum_overdue_order_past_30_day
3. sum_current_normal_order
4. total_loan_prin_amt
5. sum_current_order_15

In [17]:
# Finding out applied date = last loan disbursed date for each customer. 

pan_df = df.groupby("borrower_pan")['loan_disbursement_upd_dte'].max()
pan_df = pan_df.reset_index()
pan_df['last_applied_date'] = pan_df['loan_disbursement_upd_dte']
pan_df = pan_df.drop('loan_disbursement_upd_dte', 1)
df_pan = pd.merge(df, pan_df, on='borrower_pan', how='left')

In [18]:
### total_loan_count

# creating a loan count coloumn which populates as to how many unique loans have b een taken by each customers. 

df_pan['loan_count'] = 0

for i in range(len(df_pan['loan_disbursement_upd_dte'].index)):
    if df_pan['last_applied_date'].loc[i] - timedelta(days=30) <= df_pan['loan_disbursement_upd_dte'].loc[i] and df_pan['loan_disbursement_upd_dte'].loc[i] < df_pan['last_applied_date'].loc[i]:
        df_pan['loan_count'].loc[i] = 1
    else:
        df_pan['loan_count'].loc[i] = 0

# gropping on each borrower pan on customer level and trying to find the total loan taken in pasgt 30 days

loan_count_df = df_pan.groupby("borrower_pan")['loan_count'].sum()
loan_count_df = loan_count_df.reset_index()
df_pan_1 = pd.merge(df_pan, loan_count_df, on='borrower_pan', how='left')

# Renaming and dropping flag column.

df_pan_1 = df_pan_1.drop('loan_count_x',1)
df_pan_1 = df_pan_1.rename(columns={'loan_count_y':'total_loan_count'})

In [19]:
### sum_overdue_order_past_30_day. 

 
df_pan_1['overdue_order_past_30_day_flag'] = 0

for i in range(len(df_pan_1['overdue_order_past_30_day_flag'].index)):
    if (df_pan_1['repayment_flag'].loc[i] == 'Full' or df_pan_1['repayment_flag'].loc[i] == 'Partial') and df_pan_1['final_dpd'].loc[i] > 0 and (df_pan_1['last_applied_date'].loc[i] - df_pan_1['repayment_date'].loc[i]).days <= 30 :
        df_pan_1['overdue_order_past_30_day_flag'].loc[i] = 1
    elif df_pan_1['repayment_flag'].loc[i] == 'Partial' and df_pan_1['final_dpd'].loc[i] > 0:
        df_pan_1['overdue_order_past_30_day_flag'].loc[i] = 1
    else:
        df_pan_1['overdue_order_past_30_day_flag'].loc[i] = 0

# sum of overdues on customer or Pan level.

overdue_30_df = df_pan_1.groupby("borrower_pan")['overdue_order_past_30_day_flag'].sum()
overdue_30_df = overdue_30_df.reset_index()
df_pan_2 = pd.merge(df_pan_1, overdue_30_df, on='borrower_pan', how='left')

# Renaming and dropping flag column.

df_pan_2 = df_pan_2.drop('overdue_order_past_30_day_flag_x',1)
df_pan_2 = df_pan_2.rename(columns={'overdue_order_past_30_day_flag_y':'sum_overdue_order_past_30_day'})

In [20]:
# sum_current_normal_order.. 

df_pan_2['current_normal_order_flag'] = 0

for i in range(len(df_pan_2['current_normal_order_flag'].index)):
    if (df_pan_2['repayment_flag'].loc[i] == 'Unpaid' or df_pan_2['repayment_flag'].loc[i] == 'Partial') and df_pan_2['final_dpd'].loc[i]<=0:
        df_pan_2['current_normal_order_flag'].loc[i] = 1
    else:
        df_pan_2['current_normal_order_flag'].loc[i] = 0

# sum current_normal_order on customer or Pan level.

current_normal_order_df = df_pan_2.groupby("borrower_pan")['current_normal_order_flag'].sum()
current_normal_order_df = current_normal_order_df.reset_index()
df_pan_3 = pd.merge(df_pan_2, current_normal_order_df, on='borrower_pan', how='left')

# Renaming and dropping flag column.

df_pan_3 = df_pan_3.drop('current_normal_order_flag_x',1)
df_pan_3 = df_pan_3.rename(columns={'current_normal_order_flag_y':'sum_current_normal_order'})

In [21]:
### total_loan_prin_amt

total_loan_amt_df = df_pan_2.groupby("borrower_pan")['loan_prin_amt'].sum()
total_loan_amt_df = total_loan_amt_df.reset_index()
df_pan_4 = pd.merge(df_pan_3, total_loan_amt_df, on='borrower_pan', how='left')

# Renaming the total_loan_print_amt column. 
df_pan_4 = df_pan_4.rename(columns={'loan_prin_amt_y':'total_loan_prin_amt'})

In [22]:
# sum_current_order_15+_days..


df_pan_4['current_order_15+_days_flag'] = 0

for i in range(len(df_pan_4['current_order_15+_days_flag'].index)):
    if (df_pan_4['repayment_flag'].loc[i] == 'Unpaid' or df_pan_4['repayment_flag'].loc[i] == 'Partial') and df_pan_4['final_dpd'].loc[i] > 15:
        df_pan_4['current_order_15+_days_flag'].loc[i] = 1
    else:
        df_pan_4['current_order_15+_days_flag'].loc[i] = 0

# sum_current_order_15+_days on customer level. 

sum_current_order_15plus_days_df = df_pan_4.groupby("borrower_pan")['current_order_15+_days_flag'].sum()
sum_current_order_15plus_days_df = sum_current_order_15plus_days_df.reset_index()
df_pan_5 = pd.merge(df_pan_4, sum_current_order_15plus_days_df, on='borrower_pan', how='left')

# Renaming and dropping flag column.

df_pan_5 = df_pan_5.drop('current_order_15+_days_flag_x',1)
df_pan_5 = df_pan_5.rename(columns={'current_order_15+_days_flag_y':'sum_current_order_15+_days'})

### Calculation of metrics related to Whitelist API.

1. Repay_past_7Day_Cust_flag
2. total_unpaid_amt
3. Tot_Unpaid_Amt_LT10K_Cust_Flag
4. unpaid_overdue_order_flag
5. Total_unpaid_overdue_order
6. None_Unpaid_Overdue_Order_Cust_Flag
7. Repay_Due_over_3DPD_Past_14_Days_Loan_flag
8. 

In [23]:
# Repay_past_7Day_Cust_flag

df_pan_5['Repay_past_7Day_Cust_flag'] = 0

for i in range(len(df_pan_5['Repay_past_7Day_Cust_flag'].index)):
    if (df_pan_5['current_date'].loc[i] - df_pan_5['repayment_date'].loc[i]).days <= 7:
        df_pan_5['Repay_past_7Day_Cust_flag'].loc[i] = 1
    else:
        df_pan_5['Repay_past_7Day_Cust_flag'] = 0        

# sum_repay_past_7Day_Cust on customer level. 
        
Repay_past_7Day_Cust_flag_df = df_pan_5.groupby("borrower_pan")['Repay_past_7Day_Cust_flag'].sum()
Repay_past_7Day_Cust_flag_df = Repay_past_7Day_Cust_flag_df.reset_index()
df_pan_6 = pd.merge(df_pan_5, Repay_past_7Day_Cust_flag_df, on='borrower_pan', how='left')

# Renaming and dropping flag column.

df_pan_6 = df_pan_6.drop('Repay_past_7Day_Cust_flag_x',1)
df_pan_6 = df_pan_6.rename(columns={'Repay_past_7Day_Cust_flag_y':'sum_repay_past_7Day_Cust'})

In [35]:
# Total Unpaid amount 

Unpaid_amt_df = df_pan_6[(df_pan_6['repayment_flag'] == 'Partial') | (df_pan_6['repayment_flag'] == 'Unpaid')].groupby("borrower_pan")['final_due_amt'].sum()
Unpaid_amt_df = Unpaid_amt_df.reset_index()
df_pan_7 = pd.merge(df_pan_6, Unpaid_amt_df, on='borrower_pan', how='left')

# Renaming original and new column.
df_pan_7 = df_pan_7.rename(columns={'final_due_amt_x':'final_due_amt'})
df_pan_7 = df_pan_7.rename(columns={'final_due_amt_y':'total_unpaid_amt'})
df_pan_7 = df_pan_7.rename(columns={'loan_prin_amt_x':'loan_prin_amt'})

# Filling the null values in total_unpaid_amt.
df_pan_7['total_unpaid_amt'] = df_pan_7['total_unpaid_amt'].fillna(0)

#################################################################################

# Total Unpaid amount fkag: Tot_Unpaid_Amt_LT10K_Cust_Flag
# this specifies which customers are having a total unpaid amount which is less than 10k. 

df_pan_7['Tot_Unpaid_Amt_LT10K_Cust_Flag'] = 0

for i in range(len(df_pan_7['total_unpaid_amt'].index)):
    if df_pan_7['total_unpaid_amt'].loc[i] < 10000:
        df_pan_7['Tot_Unpaid_Amt_LT10K_Cust_Flag'].loc[i] = 1
    else:
        continue

In [52]:
# unpaid overdue order. 
## unpaid_overdue_order_flag. 

df_pan_7['unpaid_overdue_order_flag'] = 0

for i in range(len(df_pan_7['unpaid_overdue_order_flag'].index)):
    if (df_pan_7['repayment_flag'].loc[i] == 'Partial' or df_pan_7['repayment_flag'].loc[i] == 'Unpaid') and (df_pan_7['final_dpd_new'].loc[i]>0):
        df_pan_7['unpaid_overdue_order_flag'] = 1
    else:
        continue

##################################################################

# Total_unpaid_overdue_order....

unpaid_overdue_order_df = df_pan_7.groupby("borrower_pan")['unpaid_overdue_order_flag'].sum()
unpaid_overdue_order_df = unpaid_overdue_order_df.reset_index()
df_pan_8 = pd.merge(df_pan_7, unpaid_overdue_order_df, on='borrower_pan', how='left')

# Renaming exisisting and new column.

df_pan_8 = df_pan_8.rename(columns={'unpaid_overdue_order_flag_x':'unpaid_overdue_order_flag'})
df_pan_8 = df_pan_8.rename(columns={'unpaid_overdue_order_flag_y':'Total_unpaid_overdue_order'})


##################################################################


# creating None_Unpaid_Overdue_Order_Cust_Flag

df_pan_8['None_Unpaid_Overdue_Order_Cust_Flag'] = 0

for i in range(len(df_pan_8['None_Unpaid_Overdue_Order_Cust_Flag'].index)):
    if df_pan_8['Total_unpaid_overdue_order'].loc[i] == 0:
        df_pan_8['None_Unpaid_Overdue_Order_Cust_Flag'].loc[i] = 1 
    else:
        continue

In [ ]:
#### Repay_Due_over_3DPD_Past_14_Days_Loan_flag...

df_pan_8['Repay_Due_over_3DPD_Past_14_Days_Loan_flag'] = 0

for i in range(len(df_pan_8['Repay_Due_over_3DPD_Past_14_Days_Loan_flag'].index)):
    if (df_pan_8['repayment_flag'].loc[i] == 'Partial' or df_pan_8['repayment_flag'].loc[i] == 'Unpaid') and (df_pan_8['final_dpd_new'].loc[i]>3) and ((df_pan_8['current_date']-df_pan_8['loan_disbursement_upd_dte'].days) <= 14):
        df_pan_8['Repay_Due_over_3DPD_Past_14_Days_Loan_flag'].loc[i] = 1
    else:
        continue 

In [69]:
# Total_Repay_Due_over_3DPD_Past_14_Days_Loan....

Repay_Due_over_3DPD_Past_14_Days_Loan_df = df_pan_7.groupby("borrower_pan")['unpaid_overdue_order_flag'].sum()
unpaid_overdue_order_df = unpaid_overdue_order_df.reset_index()
df_pan_8 = pd.merge(df_pan_7, unpaid_overdue_order_df, on='borrower_pan', how='left')

,partner_loan_id_comb,borrower_pan,loan_prin_amt,loan_proc_fee,loan_amt_disbursed,loan_int_amt,loan_emi_dte_1,loan_end_dte,loan_disbursement_upd_dte,emi_amt,old_partnerid,repayment_date,repayment_amt,current_date,initial_DPD,initial_DPD_new,DPD_Amount,capped_dpd_amt,expected_amt,total_repaid_amt,due_amt,repayment_flag,final_dpd,final_dpd_new,final_dpd_amt,final_capped_dpd_amt,final_expected_amt,final_due_amt,last_applied_date,total_loan_count,sum_overdue_order_past_30_day,sum_current_normal_order,total_loan_prin_amt,sum_current_order_15+_days,sum_repay_past_7Day_Cust,total_unpaid_amt,Tot_Unpaid_Amt_LT10K_Cust_Flag,unpaid_overdue_order_flag,Total_unpaid_overdue_order,None_Unpaid_Overdue_Order_Cust_Flag
0,KUD-BTH-781116899997182949199636,BAMPC1949R,2100.0,545.16,1554.84,13.23,2020-11-29,2020-12-05,2020-11-29,2113.23,KUD-BTH-78111,NaT,0.0,2020-12-10,5,5,105.0,105.0,2218.23,0.0,2218.23,Unpaid,5,5,105.0,105.0,2218.23,2218.23,2020-11-29,0,0,0,2100.0,0,0,2218.23,1,1,1,0
1,KUD-BTH-781116900112694658520768,BPJPM5784E,1800.0,467.28,1332.72,11.34,2020-11-29,2020-12-05,2020-11-29,1811.34,KUD-BTH-78111,NaT,0.0,2020-12-10,5,5,90.0,90.0,1901.34,0.0,1901.34,Unpaid,5,5,90.0,90.0,1901.34,1901.34,2020-11-29,0,0,0,1800.0,0,0,1901.34,1,1,1,0
2,KUD-BTH-781116900154773213910750,BCYPK0940G,1800.0,467.28,1332.72,11.34,2020-11-29,2020-12-05,2020-11-29,1811.34,KUD-BTH-78111,NaT,0.0,2020-12-10,5,5,90.0,90.0,1901.34,0.0,1901.34,Unpaid,5,5,90.0,90.0,1901.34,1901.34,2020-11-29,0,0,0,1800.0,0,0,1901.34,1,1,1,0
3,KUD-BTH-781116900155739556417298,FWIPS4293A,4300.0,1116.28,3183.72,27.09,2020-11-29,2020-12-05,2020-11-29,4327.09,KUD-BTH-78111,NaT,0.0,2020-12-10,5,5,215.0,215.0,4542.09,0.0,4542.09,Unpaid,5,5,215.0,215.0,4542.09,4542.09,2020-11-29,6,0,0,51900.0,0,0,22831.04,0,1,12,0
4,KUD-BTH-781116900171038510913233,BRTPD0471R,1800.0,467.28,1332.72,11.34,2020-11-29,2020-12-05,2020-11-29,1811.34,KUD-BTH-78111,NaT,0.0,2020-12-10,5,5,90.0,90.0,1901.34,0.0,1901.34,Unpaid,5,5,90.0,90.0,1901.34,1901.34,2020-11-29,0,0,0,1800.0,0,0,1901.34,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196750,KUD-BTH-781116873522043243665050,BAEPJ8551L,2500.0,590.00,1910.00,17.50,2020-09-17,2020-09-23,2020-09-17,2517.50,KUD-BTH-78111,2020-09-22,2515.0,2020-12-10,-1,0,0.0,0.0,2517.50,2515.0,2.50,Full,-1,0,0.0,0.0,2517.50,2.50,2020-11-23,3,2,0,34800.0,2,0,13499.04,0,1,10,0
196751,KUD-BTH-781116873522124975986198,AHRPN6729P,4000.0,944.00,3056.00,32.00,2020-09-17,2020-09-24,2020-09-17,4032.00,KUD-BTH-78111,2020-09-24,4032.0,2020-12-10,0,0,0.0,0.0,4032.00,4032.0,0.00,Full,0,0,0.0,0.0,4032.00,0.00,2020-10-12,9,2,0,32000.0,3,0,12863.50,0,1,10,0
196752,KUD-BTH-781116873522215174493723,KXOPS1157R,3500.0,826.00,2674.00,24.50,2020-09-17,2020-09-23,2020-09-17,3524.50,KUD-BTH-78111,2020-09-22,3521.0,2020-12-10,-1,0,0.0,0.0,3524.50,3521.0,3.50,Full,-1,0,0.0,0.0,3524.50,3.50,2020-09-22,1,0,0,7000.0,1,0,5190.50,1,1,2,0
196753,KUD-BTH-781116873524092035845906,GDYPK8157F,3500.0,826.00,2674.00,24.50,2020-09-17,2020-09-23,2020-09-17,3524.50,KUD-BTH-78111,NaT,0.0,2020-12-10,78,78,1701.0,1701.0,5225.50,0.0,5225.50,Unpaid,78,78,1701.0,1701.0,5225.50,5225.50,2020-09-17,0,0,0,3500.0,1,0,5225.50,1,1,1,0
